In [122]:
import os
from pymongo import MongoClient
import pandas as pd
import json

In [18]:
startmongo = 'service mongod start'
command = 'mongoimport --db companies --collection companies --file Database/companies.json'

os.popen(startmongo)
os.popen('service mongod status')
os.popen(command)

In [109]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [110]:
db,coll = (list(connectCollection('companies','companies')))

In [123]:
#q = {'$or':[{"offices.latitude":{'$ne':'null'}},{"offices.longitude":{'$ne':'null'}}]}
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))

'''name = rest[0]['name']
founded_year = rest[0]['founded_year']
category = rest[0]['category_code']
latitude = rest[0]['offices'][0]['latitude']
longitude = rest[0]['offices'][0]['longitude']
value = rest[0]['ipo']['valuation_amount']
'''
clean_db = []
for i in range(len(rest)):
    count=0
    diction ={
        'name':rest[i]['name'],
        'category':rest[i]['category_code'],
        #'deadpooled':rest[i]['deadpooled_year'],
        'value': rest[i].get('total_money_raised',0)
        
    }
    for ind in range(len(rest[i]['offices'])):
        if rest[i]['offices']:
            diction[f'Location-{ind+1}'] = {
                'type':'Point',
                'coordinates':[float(rest[i]['offices'][ind]['longitude']), float(rest[i]['offices'][ind]['latitude'])]
            }
        
    
    clean_db.append(diction)
clean_db
df = pd.DataFrame(clean_db)
    

In [124]:
df.head()

,name,category,value,Location-1,Location-2,Location-3,Location-4,Location-5,Location-6,Location-7,Location-8,Location-9,Location-10,Location-11,Location-12
0,Postini,web,$0,"{'type': 'Point', 'coordinates': [-122.247573,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Digg,news,$45M,"{'type': 'Point', 'coordinates': [-122.394523,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Geni,web,$16.5M,"{'type': 'Point', 'coordinates': [-118.393064,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Flektor,games_video,$0,"{'type': 'Point', 'coordinates': [-118.379768,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fox Interactive Media,web,$0,"{'type': 'Point', 'coordinates': [-118.39417, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
with open('./Database/database_clean.json', 'w') as fp:
    json.dump(clean_db, fp)